In [146]:
import csv
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import itertools as it
from loguru import logger
import time
# this is where any files or directories/path variables go:
csv_filename = './meetings.csv'


## SETUP A LOGURU LOGGER:
<hr>

In [147]:
log_file_path = './data/meeting_scraper.log'
rotation="100 MB"

In [148]:
logger.add(log_file_path, rotation=rotation, level="INFO", format="[<green>{time: MMM D YYYY HH:mm:ss:SSSS}</>] | <level>{message}</>", backtrace=True, diagnose=True)

8

## BUILDING A BASIC SCRAPER:
<hr>

In [149]:
# CREATE A CSV PARSER TO GET LINKS FROM CSV FILE:
def csv_parser(csv_reader, header: str):
    _header = next(csv_reader)
    headerIndex = _header.index(header)
    # now create an empty list to append the addresses to
    data_list = []
    # loop through CSV file and append to address_list
    for line in csv_reader:
        all_data = line[headerIndex]
        data_list.append(all_data)
    return data_list


In [150]:
# create func to get list of links:
def get_links(csv_filename):
    with open(csv_filename, 'r') as f:
        csv_reader = csv.reader(f)
        link_list = csv_parser(csv_reader, 'link')
    return link_list

# create func to get list of links:
def get_addresses(csv_filename):
    with open(csv_filename, 'r') as f:
        csv_reader = csv.reader(f)
        address_list = csv_parser(csv_reader, 'address')
    return address_list

In [151]:
def fetch_soup_data(link):
    # STEP 1: Get the webpage response obj from requests
    page = requests.get(link)
    # STEP 2: Get Soup object for html parsing
    soup = bs(page.text, "lxml")
    logger.info("got soup data for link: {}", link)
    return soup
    

In [152]:
# CREATE A FUNCTION THAT WILL EXTRACT ADDRESS DATA FROM EACH LINK IN 
# LINK LIST:
def get_address_data(soup):
    
    try:
        address_tag = soup.address
        address = address_tag.contents[1]
        
        meeting_name = soup.find(
            'div', class_='fui-card-body').find(class_='weight-300')
        name = meeting_name.contents[1]
        
        city_tag = meeting_name.find_next('a')
        city = city_tag.contents[0]
        
        state_tag = city_tag.find_next('a')
        state = state_tag.contents[0]
        logger.info("[+] Address data retrieved")
        return {'name': name, 'address': address, 'city': city, 'state': state}
        
    except IndexError as ie:
        logger.error("[-] UnboundError occured {} ", ie)
        try:
            return {'name': name, 'address': address, 'city': city, 'state': 'state'}
        except UnboundLocalError as ule:
            logger.error("[-] UnboundLocalError occured: {} ", ule)
        try:
            return {'name': name, 'address': address, 'city': 'city', 'state': state}
        except UnboundLocalError as ule:
            logger.error("[-] UnboundError occured {} ", ule)
        try:
            return {'name': name, 'address': 'address', 'city': city, 'state': state}
        except UnboundLocalError as ule:
            logger.error("[-] UnboundError occured {} ", ule)
        try:
            return {'name': 'name', 'address': address, 'city': city, 'state': state}
        except UnboundLocalError as ule:
            logger.error("[-] UnboundError occured {} ", ule)

In [153]:
# CREATE A FUNCTION THAT WILL EXTRACT ALL THE TABLE DATA FROM EACH LINK
# IN THE LINK LIST. THE TABLE DATA WILL THEN NEED TO BE PARSED AND 
# CLEANED IF THERE ARE MULTIPLE ITEMS:
def get_table_data(soup):
    try:
        info_table = soup.find('table', class_='table fui-table')
        # obtain all the columns from <th>
        headers = []
        for i in info_table.find_all('th'):
            title = i.text
            headers.append(title.lower())

            # now create a dataframe:
        df = pd.DataFrame(columns=headers)

    
        # Now create the foor loop to fill dataframe
        # a row is under the <tr> tags and data under the <td> tags
        for j in info_table.find_all('tr')[1:]:
            # if info_table.find_all('tr')[1:] == AttributeError.NoneType:
            #     print("No info table found")
            row_data = j.find_all('td')
            row = [i.text for i in row_data]
            length = len(df)
            df.loc[length] = row

        # data['day'].append(df['day'].to_list())
        # data['time'].append(df['time'].to_list())
        # data['info'].append(df['info'].to_list())
        day = df['day'].to_list()
        time = df['time'].to_list()
        info = df['info'].to_list()

        logger.info("[+] Meeting Details Retrieved")
        # now return data
        return {'day': day, 'time': time, 'info': info}
    
    except AttributeError as ae:
        logger.error("info_table.find_all('tr') failed: {}", ae)
        return {'day': 'day', 'time': 'time', 'info': 'info'}

In [154]:
# CREATE A FUNCTION THAT WILL PARSE THE ROW DATA AND STORE IT 
# IN A DICTIONARY. THAT DICTIONARY CAN THEN BE INSERTED INTO 
# A LIST OF DICTIONARIES CALLED ROW_LIST BUT TO DO THIS THE PARSER
# HAS TO JOIN LIST ITEMS INTO ONE LONG STRING SO EACH ROW HAS THE 
# SAME NUMBER OF COLUMNS:
# THIS WAS INSERTED INTO meeting_data_scraper

# VERSION 1
def row_parser1(item0, item1):
    """
    :param item0: This is the address data in a dictionary. Use the following keys to access
    the data -> Keys: 'name' - 'address' - 'city' - 'state' 
    :param item1: This is the meeting details data in a dictionary. Use the following keys to
    access the data -> Keys: 'day' - 'time' - 'info'
    
    create a final dictionary that will be used to store the information in the database as one row. 
    I will need to join the list items to create one string with a | seperating each item so I can 
    split the string when retrieving the data.
    """
    row = {}
    try:
        row['name'] = item0['name']
        row['address'] = item0['address']
        row['city'] = item0['city']
        row['state'] = item0['state']
        logger.info("[+] Row Data Parsed")
    except Exception as e:
        logger.error("[-] Row Data Raised Exception: {}", e)
        print(e)
        row['name'] = 'name'
        row['address'] = 'address'
        row['city'] = 'city'
        row['state'] = 'state'

    # now add item1 to the row data
    row['day'] = ' | '.join(item1['day'])
    row['time'] = ' | '.join(item1['time'])
    row['info'] = ' | '.join(item1['info'])

    # now return the row data dictionary
    
    return row

# VERSION 2: 
def row_parser2(item0, item1):
    """
    :param item0: This is the address data in a dictionary. Use the following keys to access
    the data -> Keys: 'name' - 'address' - 'city' - 'state' 
    :param item1: This is the meeting details data in a dictionary. Use the following keys to
    access the data -> Keys: 'day' - 'time' - 'info'
    
    create a final dictionary that will be used to store the information in the database as one row. 
    I will need to join the list items to create one string with a | seperating each item so I can 
    split the string when retrieving the data.
    """
    try:
        row = {}
        try:
            row['name'] = item0['name']
            row['address'] = item0['address']
            row['city'] = item0['city']
            row['state'] = item0['state']
            # now add item1 to the row data
            row['day'] = ' | '.join(item1['day'])
            row['time'] = ' | '.join(item1['time'])
            row['info'] = ' | '.join(item1['info'])
            # now return the row data dictionary
            return row
        except Exception as e:
            print(f'{e}')
    except Exception as e:
        print(f'{e}')
        for k, v in row.items():
            if v is not None:
                pass
            else:
                v = k
        return row


In [155]:
# THIS IS THE 'MAIN LOGICAL FUNCTION' THIS FUNCTION WILL COMBINE THE 
# get_address_data, get_table_data, and meeting_row_parser FUNCTIONS.
# THAT WAY I CAN EXECUTE ALL OF THE FUNCTIONS IN ONE CALL.
def meeting_data_scraper(link):
    
    # Get Soup Data
    soup = fetch_soup_data(link)
    # Create two dicts with the following keys
    address_dict = get_address_data(soup)
    details_dict = get_table_data(soup)

    d = [address_dict, details_dict]
    row_data = row_parser1(d[0], d[1])

    logger.info("[+] Main Meeting Dart Scraper Worked")
    return row_data


In [156]:
def flatten(list_of_lists):
    "Flatten one level of nesting"
    return it.chain.from_iterable(list_of_lists)

<hr>

In [157]:
# STEP 1: CREATE A CSV PARSER TO GET LINKS FROM CSV FILE:
def csv_parser(csv_reader, header: str):
    """
    :param csv_reader: This is a csv_reader object from the csv package
    :param header: This is the header of the CSV file
    :returns: a list of all the data from the CSV file row for row:
    """
    _header = next(csv_reader)
    headerIndex = _header.index(header)
    # now create an empty list to append the addresses to
    data_list = []
    # loop through CSV file and append to address_list
    for line in csv_reader:
        all_data = line[headerIndex]
        data_list.append(all_data)
    return data_list

# THIS FUNC USES THE csv_parser TO EXTRACT LINKS FROM THE meetings.csv file:
def get_links(csv_filename):
    """
    :param csv_filename: This is where the CSV file path that you want to read from goes
    :returns: a list of links
    """
    with open(csv_filename, 'r') as f:
        csv_reader = csv.reader(f)
        link_list = csv_parser(csv_reader, 'link')
    return link_list

# create func to get list of links:
def get_addresses(csv_filename):
    """
    :param csv_filename: This is where the CSV file path that you want to read from goes
    :returns: a list of addresses
    """
    with open(csv_filename, 'r') as f:
        csv_reader = csv.reader(f)
        address_list = csv_parser(csv_reader, 'address')
    return address_list


def fetch_soup_data(link):
    """
    Basic function that takes a link from link_list and accepts it as an argument, 
    then uses the requests.get method to return an html page. Then uses BeautifulSoup 
    to parse the pages' text using lxml
    """
    # STEP 1: Get the webpage response obj from requests
    page = requests.get(link)
    # STEP 2: Get Soup object for html parsing
    soup = bs(page.text, "lxml")
    logger.info("got soup data for link: {}", link)
    return soup

In [158]:
# CREATE A FUNCTION THAT WILL EXTRACT ADDRESS DATA FROM EACH LINK IN 
# LINK LIST:
def get_address_data(soup):
    """
    :description: Extracts the address data from a soup object that gets past to it in a 
    for loop. 

    :param soup: This is a BeautifulSoup object that has been pulled from a soup_data list that was 
    created using the fetch_soup_data function and stored into a soup_data list.

    :returns: A dictionary containing the address data components: {address, name, city, state}

    ::: I need to figure out how to add zip_code to this :::
    """
    try:
        address_tag = soup.address
        address = address_tag.contents[1]
        
        meeting_name = soup.find(
            'div', class_='fui-card-body').find(class_='weight-300')
        name = meeting_name.contents[1]
        
        city_tag = meeting_name.find_next('a')
        city = city_tag.contents[0]
        
        state_tag = city_tag.find_next('a')
        state = state_tag.contents[0]
        logger.info("[+] Address data retrieved")
        return {'name': name, 'address': address, 'city': city, 'state': state}
        
    except IndexError as ie:
        logger.error("[-] UnboundError occured {} ", ie)
        try:
            return {'name': name, 'address': address, 'city': city, 'state': 'state'}
        except UnboundLocalError as ule:
            logger.error("[-] UnboundLocalError occured: {} ", ule)
        try:
            return {'name': name, 'address': address, 'city': 'city', 'state': state}
        except UnboundLocalError as ule:
            logger.error("[-] UnboundError occured {} ", ule)
        try:
            return {'name': name, 'address': 'address', 'city': city, 'state': state}
        except UnboundLocalError as ule:
            logger.error("[-] UnboundError occured {} ", ule)
        try:
            return {'name': 'name', 'address': address, 'city': city, 'state': state}
        except UnboundLocalError as ule:
            logger.error("[-] UnboundError occured {} ", ule)


# CREATE A FUNCTION THAT WILL EXTRACT ALL THE TABLE DATA FROM EACH LINK
# IN THE LINK LIST. THE TABLE DATA WILL THEN NEED TO BE PARSED AND 
# CLEANED IF THERE ARE MULTIPLE ITEMS:
def get_table_data(soup):
    """
    :description: Extracts the meeting details data from a soup object that gets past to it in a 
    for loop. The data is extracted from an html table. Each row that returns a list is joined together to make 
    one string so it can be saved to a csv file.

    :param soup: This is a BeautifulSoup object that has been pulled from a soup_data list that was 
    created using the fetch_soup_data function and stored into a soup_data list.

    :returns: a dictionary with the days the meeting runs, the time, and info about the meeting: 
    {day, time, info}
    """
    try:
        info_table = soup.find('table', class_='table fui-table')
        # obtain all the columns from <th>
        headers = []
        for i in info_table.find_all('th'):
            title = i.text
            headers.append(title.lower())

            # now create a dataframe:
        df = pd.DataFrame(columns=headers)

    
        # Now create the foor loop to fill dataframe
        # a row is under the <tr> tags and data under the <td> tags
        for j in info_table.find_all('tr')[1:]:
            # if info_table.find_all('tr')[1:] == AttributeError.NoneType:
            #     print("No info table found")
            row_data = j.find_all('td')
            row = [i.text for i in row_data]
            length = len(df)
            df.loc[length] = row

        # data['day'].append(df['day'].to_list())
        # data['time'].append(df['time'].to_list())
        # data['info'].append(df['info'].to_list())
        day = df['day'].to_list()
        time = df['time'].to_list()
        info = df['info'].to_list()

        logger.info("[+] Meeting Details Retrieved")
        # now return data
        return {'day': day, 'time': time, 'info': info}
    
    except AttributeError as ae:
        logger.error("info_table.find_all('tr') failed: {}", ae)
        return {'day': 'day', 'time': 'time', 'info': 'info'}
        

In [159]:
def row_parser(item0, item1):
    """
    :param item0: This is the address data in a dictionary. Use the following keys to access
    the data -> Keys: 'name' - 'address' - 'city' - 'state' 
    :param item1: This is the meeting details data in a dictionary. Use the following keys to
    access the data -> Keys: 'day' - 'time' - 'info'
    
    create a final dictionary that will be used to store the information in the database as one row. 
    I will need to join the list items to create one string with a | seperating each item so I can 
    split the string when retrieving the data.
    """
    row = {}
    try:
        row['name'] = item0['name']
        row['address'] = item0['address']
        row['city'] = item0['city']
        row['state'] = item0['state']
        row['zip_code'] = ''
        logger.info("[+] Row Data Parsed")
    except Exception as e:
        logger.error("[-] Row Data Raised Exception: {}", e)
        print(e)
        row['name'] = 'name'
        row['address'] = 'address'
        row['city'] = 'city'
        row['state'] = 'state'

    # now add item1 to the row data
    row['day'] = ' | '.join(item1['day'])
    row['time'] = ' | '.join(item1['time'])
    row['info'] = ' | '.join(item1['info'])

    # now return the row data dictionary
    
    return row



In [160]:
new_csv_file = './meeting_details.csv'
headers = ["name", 'address', 'city', 'state', 'zip_code', 'day', 'time', 'info']
def csv_writer(row_data, csv_filename, headers=None):
    with open(csv_filename, 'a') as f:
        if headers is not None:
            writer = csv.DictWriter(f, fieldnames=headers, delimiter='|')
            writer.writeheader(headers)
            for row in row_data:
                writer.writerow(row)
        else:
            fieldnames = [k for k in row_data.keys()]
            writer = csv.DictWriter(f, fieldnames=fieldnames, delimiter='|')
            writer.writeheader(fieldnames)
            for row in row_data:
                writer.writerow(row)

#### BASIC COMPONENTS BUILT..

<hr>

## CREATE LINK LISTS:

In [161]:
# whole list of links
link_list = get_links(csv_filename)

# shortened list: first 1000
links_1000 = link_list[:1000]

# shortened list: first 500
links_500 = link_list[:500]

links_100 = link_list[:100]

####################################################################################
# chunk the links into bunches of 10,000
link_list1 = link_list[:10000]
link_list2 = link_list[10000:20000]
link_list3 = link_list[20000:30000]
link_list4 = link_list[30000:]

print(len(link_list1))
print(len(link_list2))
print(len(link_list3))
print(len(link_list4))


10000
10000
10000
3588


##### BUILD TWO FUNCTIONS PUTTING ALL THE ABOVE COMPONENTS TOGETHER
<hr>

In [162]:
def soup_data_scraper(soup_data):
    """
    :param soup_data: This is the list of soup data that was returned by the scrape function. 

    :returns: a list of dictionaries that will get stored to the variable row since each dict 
    creates a row of data for the csv file. 
    """
    try:
        rows = []
        for soup in soup_data:
            # Create two dicts with the following keys
            address_dict = get_address_data(soup)
            details_dict = get_table_data(soup)
            logger.info("[+] Address and table data parsed")
            d = [address_dict, details_dict]
            row_data = row_parser(d[0], d[1])
            rows.append(row_data)
        logger.info("++++++++ SCRAPED ALL THE LINKS +++++++++ ")
        return rows

    except IndexError as ie:
        logger.error("{}: List Exhausted. No more Soup Items", ie)

    except Exception as e:
        logger.error("{}: Exception raised", e)

In [163]:
def get_rows(link_list):
    """
    :param link_list: The list that is returned from the get_links function. Then it is passed to 
    scrape function that is used inside this function. 

    :returns: a list of dictionaries that will get stored to the variable row since each dict 
    creates a row of data for the csv file. 
    """
    try:
        soup_data = scrape(link_list)
        rows = soup_data_scraper(soup_data)
        return rows

    except requests.exceptions.RequestException as re:
        time.sleep(10)
        logger.error("{} Request Exception occured", re)
        
    except Exception as e:
        logger.error("{}: Exception while scraping link list", e)

new_csv_filename = './meeting_details.csv'

def scrape(link_list, csv_file):
    """
    :param link_list: A list of links to scrape and get the soup data from the site, 
    parse the data and append the soup data to the soup_data list. 
    """
    # Two empty lists. One for soup_data and another to store row_data
    row_data = []
    soup_data = []
    # The headers for the csv_file
    headers = ["name", 'address', 'city', 'state', 'zip_code', 'day', 'time', 'info']
    try:
        count = 0
        for link in link_list:
            soup = fetch_soup_data(link)
            soup_data.append(soup)
            logger.info("[+] {} scraped successfully: Link number {}", link, count)
            count += 1
            if count % 500 == 0:
                time.sleep(5)
                # Every 500 links, write the row data to the new CSV file
                for soup in soup_data:
                    row = soup_data_scraper(soup)
                    row_data.append(row)
                # after each soup item is parsed and the results appended to row_data, append row data to csv
                csv_writer(row_data, csv_file, headers)
                # Then clear both soup and row data lists
                soup_data.clear()
                row_data.clear()
                # log the success of the script
                logger.info("[+] Count is divisible by 500:{} | Writing ROW_DATA to [{}] | soup_data and row_data cleared!", count, csv_file)
        logger.info("<<<<<<<<<<<<<<< SUCCESS >>>>>>>>>>>>>>>>")
    except IndexError as e:
        logger.error("{}: List Exhausted. No more links to scrape.", e)


In [164]:
scrape(links_1000, new_csv_filename)

2022-06-11 13:58:59.343 | INFO     | __main__:fetch_soup_data:51 - got soup data for link: https://www.aa-meetings.com/aa-meeting/chillicothe-first-capital-group/
2022-06-11 13:58:59.344 | INFO     | __main__:scrape:38 - [+] https://www.aa-meetings.com/aa-meeting/chillicothe-first-capital-group/ scraped successfully: Link number 0
2022-06-11 13:58:59.700 | INFO     | __main__:fetch_soup_data:51 - got soup data for link: https://www.aa-meetings.com/aa-meeting/chillicothe-big-book-study/
2022-06-11 13:58:59.702 | INFO     | __main__:scrape:38 - [+] https://www.aa-meetings.com/aa-meeting/chillicothe-big-book-study/ scraped successfully: Link number 1
2022-06-11 13:59:00.042 | INFO     | __main__:fetch_soup_data:51 - got soup data for link: https://www.aa-meetings.com/aa-meeting/waverly-pike-county-group/
2022-06-11 13:59:00.043 | INFO     | __main__:scrape:38 - [+] https://www.aa-meetings.com/aa-meeting/waverly-pike-county-group/ scraped successfully: Link number 2
2022-06-11 13:59:00.441

TypeError: __init__() missing 1 required positional argument: 'fieldnames'

In [39]:
# Scraped to 13180
new_csv_file = './meeting_details.csv'
headers = ["name", 'address', 'city', 'state', 'zip_code', 'day', 'time', 'info']
def csv_writer(row_data, csv_filename, headers=None):
    with open(csv_filename, 'w') as f:
        if headers is not None:
            writer = csv.DictWriter(f, fieldnames=headers, delimiter='|')
            writer.writeheader(headers)
            for row in row_data:
                writer.writerow(row)
        else:
            fieldnames = [k for k in row_data.keys()]
            writer = csv.DictWriter(f, fieldnames=fieldnames, delimiter='|')
            writer.writeheader(fieldnames)
            for row in row_data:
                writer.writerow(row)

csv_writer(row_data1, csv_filename, headers)



NameError: name 'row_data' is not defined